# 01 - Information Retrieval

This notbook demonstrates how to extract paper abstracts from the MIDAS website and to store them locally for processing. Beautifulsoup (v4) is used for extracting information from web pages, and TinyDB is used as a simple means for storing and querying for papers.

Section I in this notebook demonstrates how information can be retrieved from the MIDAS website. The code from this section is encapsulated in the genscai.retrieval module. For staging data for the other example notebooks, Section II show how the genscai.retrieval module is used.

## Section 0 - Notebook Setup

In [ ]:
%pip install --upgrade --quiet beautifulsoup4

In [ ]:
%pip install --upgrade --quiet tinydb

In [ ]:
%pip install --upgrade --quiet ipywidgets # for displaying progress bars in the notebook

## Section I - Retrieving Paper Information from the MIDAS Website

Define a function for retrieving a page of filtered articles

In [ ]:
import requests

HTTP_HEADERS = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:133.0) Gecko/20100101 Firefox/133.0'
}

def retrieve_page(page_number):
    url = 'https://midasnetwork.us/wp-admin/admin-ajax.php'

    data = {
        'action': 'filter_papers',
        'paged': f'{page_number}',
        'journal': '',
        'author': '',
        'title': '',
        'startdate': '2024-11-01',
        'enddate': '',
        'displaydefault': '',
    }

    return requests.post(url, data=data, headers=HTTP_HEADERS)

Retrieve the first page of the paginated list of filtered articles. Determine the number of pages to process.

In [ ]:
from bs4 import BeautifulSoup

page = retrieve_page(1)

soup = BeautifulSoup(page.text, 'html.parser')
pages = soup.find_all('a', {'class', 'page-numbers'})

last_page = int(pages[-2].text)

print(f'pages to process: {last_page}')

## TODO fails on a single page

Create a list of links to article detail pages by iterating over the pages of filtered articles.

In [ ]:
article_links = []
i = 1

while True:
    print(f'processing page {i} of {last_page}')

    for article in soup.find_all('article'):
        link = article.find('a')
        article_links.append(link.get('href'))

    # stop if we just processed the last page
    if i == last_page:
        break

    i += 1

    print(f'retrieving page {i}')

    page = retrieve_page(i)
    soup = BeautifulSoup(page.text, 'html.parser')

Load details for each article by iterating over the list of article detail page links and loading and processing each page.

In [ ]:
import json
import re

from ipywidgets import IntProgress
from IPython.display import display

progress_bar = IntProgress(min=0, max=len(article_links))
display(progress_bar)

articles = []
print(f'procesing {len(article_links)} articles')

for link in article_links:
    resp = requests.get(link, headers=HTTP_HEADERS)
    soup = BeautifulSoup(resp.text, 'html.parser')

    data = soup.find_all('p', {'class', 'elementor-heading-title'})
    
    article = {
        'title': data[0].text,
        'abstract': data[1].text,
        'journal': data[2].text,
        'reference': data[3].text,
        'midas_authors': []
    }
    
    refs = data[3].find_all('a')
    article['link'] = refs[1].get('href')

    # extract MIDAS author names from the page
    data = soup.find_all(class_='elementor-post__title')
    for author in data:
        article['midas_authors'].append(author.text.strip())

    articles.append(article)

    progress_bar.value += 1

Store the article details in a local NoSQL (JSON) database for later re-use.

In [ ]:
from tinydb import TinyDB, Query

db = TinyDB('db.json')
table = db.table('articles')

for article in articles:
    table.insert(article)

print(f'{len(table.all())} articles stored (total)')

## Section II - Using genscai.retrieval for Retrieving Paper Information

In [ ]:
import sys
sys.path.append('../')

from genscai import retrieval, paths

In [ ]:
retriever = retrieval.MIDASRetriever(
    startdate="2024-01-01",
    enddate="2024-12-31",
    database_path=paths.data)

retriever()